# Pre-trained

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models.video import s3d
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import torch
from torchvision import transforms
from PIL import Image
import datetime as dt

In [49]:
# Function to load video frames without transformations
def load_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    success, frame = cap.read()
    while success:
        # Convert BGR (OpenCV format) to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Convert NumPy array to PIL image
        frames.append(Image.fromarray(frame))
        success, frame = cap.read()
    cap.release()
    return frames  # Return the list of PIL images


# Save pretrained model
def save_pretrained(model):
    date_time_format = '%Y-%m-%d-%H-%M-%S'
    current_date_time_dt = dt.datetime.now()
    current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)
    model_name = f'{current_date_time_string}-pretrained-model.pt'
    save_path = f'../models/pretrained/{model_name}'

    torch.save(model.state_dict(), save_path)

    print(f"Saved model to {save_path}")


# Load pretrained model 
def load_pretrained(model, model_path, check_weights=False):
    trained_model = model.load_state_dict(torch.load(model_path, weights_only=True))

    print(f'Loaded model from {model_path}')

    # Check weights
    if check_weights:
        for var_name in model.state_dict():
            print(var_name, "\t", model.state_dict()[var_name])

    return trained_model


# Function to train the model
def train_model(device, model, train_loader, criterion, optimizer, num_epochs=5):
    trained_model = model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        print("Initializing train_loader")
        for videos, labels in train_loader:
            print("Making videos, labels")
            videos, labels = videos.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(videos)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            print(f'Loss: {running_loss/len(train_loader)}, Accuracy: {100 * correct / total}%')
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}, Accuracy: {100 * correct / total}%")

    save_pretrained(trained_model)


# Function to test the model and return predictions and true labels
def evaluate_model(device, model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for videos, labels in loader:
            videos, labels = videos.to(device), labels.to(device)
            outputs = model(videos)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)


# Function to plot confusion matrix
def plot_confusion_matrix(true_labels, preds, classes):
    cm = confusion_matrix(true_labels, preds)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    plt.show()


In [50]:
max_frames = 100  # Set your desired fixed number of frames

class CustomVideoDataset(datasets.DatasetFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        frames = self.loader(path)  # Load frames as PIL images
        frames = [transform(frame) for frame in frames]  # Apply transformations here

        # Limit to max_frames
        if len(frames) > max_frames:
            frames = frames[:max_frames]
        else:
            # If there are fewer frames, pad with zeros
            frames += [torch.zeros_like(frames[0])] * (max_frames - len(frames))

        # Stack the frames
        video_tensor = torch.stack(frames)


        video_tensor = video_tensor.permute(1, 0, 2, 3)
        video_tensor = video_tensor.unsqueeze(0)

        return video_tensor.squeeze(0), target


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for video frames
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [51]:
# Load train and test datasets using DatasetFolder
train_dir = '../downloads/train'
test_dir = '../downloads/test'

train_dataset = CustomVideoDataset(root=train_dir, loader=load_video_frames, extensions=('mp4',))
test_dataset = CustomVideoDataset(root=test_dir, loader=load_video_frames, extensions=('mp4',))

# Create data loaders as before
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Load pre-trained S3D model and modify the classifier
# model = s3d(pretrained=True)
model = s3d(weights='KINETICS400_V1')
# Access the classifier layers
dropout = model.classifier[0]  # Keep the dropout layer
num_ftrs = model.classifier[1].in_channels  # Get the number of input channels to the Conv3d layer

# Replace the last Conv3d layer with a new one that has the correct number of output classes
model.classifier = nn.Sequential(
    dropout,
    nn.Conv3d(num_ftrs, len(train_dataset.classes), kernel_size=(1, 1, 1))  # Modify for dataset's number of classes
)

model = model.to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [52]:
# Train the model
train_model(device, model, train_loader, criterion, optimizer, num_epochs=1)

Initializing train_loader
Making videos, labels
Video: tensor([[[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           ...,
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

          [[ 1.2214,  0.9303,  0.5707,  ..., -0.5424, -0.5424, -0.5424],
           [-0.0629,  0.9817,  1.0331,  ..., -0.5253, -0.5424, -0.5424],
           [-0.6452, -0.4568,  0.6906,  ..., -0.5082, -0.5253, -0.5424],
           ...,
           [-1.0904, -1.1247, -1.2617,  ..., -1.2959, -1.3130, -1.1760],
           [-1.1589, -1.0904, -1.1760,  ..., -1.3473, -1.3130, -1.2103],
           [-1.1932, -1.1247, -1.0562,  ..., -1.3815, -1.3130, -1.2274]],

          [[ 1.2043,  0.8961,  0.

KeyboardInterrupt: 

In [48]:
# Load model
model_dir = '../models/pretrained'
model_name = '2024-09-29-04-06-42-pretrained-model.pt'
model_path = f'{model_dir}/{model_name}'
load_pretrained(model, model_path)

Loaded model from ../models/pretrained/2024-09-29-04-06-42-pretrained-model.pt


<All keys matched successfully>

In [ ]:
# Evaluate on training data
train_preds, train_labels = evaluate_model(model, train_loader)

In [ ]:
# Plot confusion matrix
plot_confusion_matrix(train_labels, train_preds, train_dataset.classes)

In [ ]:
# Evaluate on testing data
test_preds, test_labels = evaluate_model(model, test_loader)
plot_confusion_matrix(test_labels, test_preds, test_dataset.classes)